In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
import json
import requests
import pandas as pd
import numpy


# Load environment variables from the .env file
load_dotenv()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 


C:\Users\starm\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [28]:
pip show langchain


Name: langchainNote: you may need to restart the kernel to use updated packages.

Version: 0.2.5
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\starm\AppData\Roaming\Python\Python311\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [98]:
TEST_MAIL = 'admin@platform.com'

In [29]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [30]:
multiply(3,4)

AttributeError: 'int' object has no attribute 'parent_run_id'

## Token

In [32]:
base_url = 'https://dashboard.colorines.paitesting.com/api'

# Set the URL for the token API
url_token = base_url+"/v1/token"

# Set the data for the POST request
data = {
    "email": "sale@platform.com",
    "password": os.getenv('CONSULTANT_PASSWORD'),
    "device_name": "colorina_2",
    "role": "consultant"
}

# Send a POST request
response = requests.post(url_token, json=data)

#print(response)

# Extract the token from the response
token = response.json().get('data').get('token')

#print("Token:", token)


## Components

In [33]:
#TODO: API need to add a filter by 'is_active'

def GetProperties(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/properties"
    params = {'include': 'step,project,development', 'filter[status]': 'available'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})
    
#PREPROCESS projects
def PreprocessProjects(df):
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    
    def filter_active_steps(steps):
        return [step['step_number'] for step in steps if step['is_active']]

    # Applying the function to create the new column
    df['active_steps'] = df['steps'].apply(filter_active_steps)

    return df

#PREPROCESS Properties
def PreprocessProperties(df):
    df['project_name'] = df['project'].apply(lambda x: x.get('name'))
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    df['step_number'] = df['step'].apply(lambda x: x.get('step_number')) #numero de etapa

    def extract_total_amount(prices, currency_code):
        total_amount = None  # Initialize to None or any default value as needed
        for entry in prices:
            if entry['type'] == 'property' and entry['currency_code'] == currency_code:
                total_amount = entry['total_amount']
                break  # Exit loop once found
        return total_amount

    # Extracting total_amount_mxn and total_amount_usd
    df['total_amount_mxn'] = df['prices'].apply(lambda x: extract_total_amount(x, 'MXN'))
    df['total_amount_usd'] = df['prices'].apply(lambda x: extract_total_amount(x, 'USD'))

    return df


# Example usage

#df_properties = GetProperties(token)

def GetProjects(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/projects"
    params = {'include': 'steps,development', 'is_active': 'True'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

df_projects = GetProjects(token)

def GetDevelopments(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/developments"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

#df_devs = GetDevelopments(token)



### Sales

In [87]:
def GetSales(token: str, status: str = None, customer_id: int = None):
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"  # Ensures the server knows you expect JSON
    }
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params={}
    if status:
        params.update({"filter[status]": status})
        
    if customer_id:
        params.update({"customer_id": customer_id})
    
    try:
        response = requests.get(url, headers=headers, params=params)
        #print("Status Code:", response.status_code)  # Log the status code
        #print("Response Body:", response.text)   
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

def PreprocessSales(df):
    df['property_name'] = df['property'].apply(lambda x: x.get('alias'))

    return df

In [88]:
test_sales = sales.copy()
PreprocessSales(test_sales)

id              custom_id   channel currency  reserve_amount  \
0   55  sa_4HKug50MakfSTRruEU  Intranet      MXN            3500   
1   54  sa_zQiJaxCxPXxgTlZUVR  Intranet      MXN            3500   
2   53  sa_By6EqINFKM8FE6ds2g  Intranet      MXN            3500   
3   52  sa_rVFjcLmBK52JRAwDrD  Intranet      MXN            3500   
4   51  sa_3U29sBIAdeXQTpzNz0  Intranet      MXN            3500   
5   50  sa_rqAL8iHqVzdRYHOx9j  Intranet      MXN            3500   
6   49  sa_pUk3ANmITwlnfMyAaE  Intranet      MXN            3500   
7   48  sa_4QosUJDzTDk37u80X1  Intranet      MXN            3500   
8   47  sa_yU0nXaZ5xbQ67Yyq4J  Intranet      MXN            3500   
9   46  sa_SIE178ue0DOh6nyva8  Intranet      MXN            3500   
10  45  sa_ioBPXTUrUDUCMChe8d  Intranet      MXN            3500   
11  44  sa_fjUp1TFq5SL9AI9QeZ  Intranet      MXN            3500   
12  43  sa_Fu8Hjrx5WMl4LZIZRJ  Intranet      MXN            3500   
13  42  sa_NXRi4Rx7U9ILuyx0mK  Intranet      MXN            3500   
14  41  sa_crQiVVHViVFP15PRTz  Intranet      MXN            3500   
15  40  sa_MTL7RbbWIpUBMgeIEv  Intranet      MXN            3500   
16  39  sa_aRFFTTIO3db8lXxkCO  Intranet      MXN            3500   
17  38  sa_B1zZJlxmHOkW4zSd9k  Intranet      MXN            3500   
18  37  sa_Btmahjjh0b0dE3AvmA  Intranet      MXN            3500   
19  36  sa_Il9kKpLxL0GEr0B2s2  Intranet      MXN            3500   
20  35  sa_ym5E7AJADwSnBMea84  Intranet      MXN            3500   
21  34  sa_kr7fS5Wp5XnIYLpxzj  Intranet      MXN            3500   
22  33  sa_rZP9rZ81BhuhUpOYpW  Intranet      MXN            3500   
23  32  sa_EOdobX2635suKfL3vt  Intranet      MXN            3500   
24  31  sa_p3pGKLV03z5okuk28W  Intranet      MXN            3500   
25  30  sa_hjNsltGQjFhycpXJCf  Intranet      MXN            3500   
26  29  sa_klmxdySq5qho0HXCYy  Intranet      MXN            3500   
27  28  sa_5jDzBevYLzpEv4KR57  Intranet      MXN            3500   
28  27  sa_fgmdl4B8U05Z6Y8xQV  Intranet      MXN            3500   
29  26  sa_hQa9uP3Y7Rp3EGicmg  Intranet      MXN            3500   
30  25  sa_1okdQ1p33D2uPy7TOW  Intranet      MXN            3500   
31  24  sa_rrYvYunJWSX3Q23vDl  Intranet      MXN            3500   
32  23  sa_ERfQrM8gziDoCH249E  Intranet      MXN            3500   
33  22  sa_Ih7WpujN1pLzgpbVqC  Intranet      MXN            3500   
34  21  sa_tFiKZSLkvo84SNV17F  Intranet      MXN            3500   
35  20  sa_b6KTxT0mdHvcFVK8Jp  Intranet      MXN            3500   
36  19  sa_LfqSkO5Q53amvLNrQG  Intranet      MXN            3500   
37  18  sa_R9pRk9APHnb5hNJwC8  Intranet      MXN            3500   
38  17  sa_jO1otuxooUV2ij1NuZ  Intranet      MXN            3500   
39  16  sa_UgZYLPlCOVsy4imfce  Intranet      MXN            3500   
40  15  sa_U4BMXmplajkbmJXNN9  Intranet      MXN            3500   
41  14  sa_ios4QZMcgT9JqWRgrm  Intranet      MXN            3500   
42  13  sa_Dn22U09lGqQpObGGY4  Intranet      MXN            3500   
43  12  sa_s15Zk84BOXI4fbQm6B  Intranet      MXN            3500   
44  11  sa_aWhfDpsREdWF5J8Hsd  Intranet      MXN            3500   
45  10  sa_iPwwHQALS7UttAjPdy  Intranet      MXN            3500   
46   9  sa_EXMx7fQbyQ7j6U9ZkL  Intranet      MXN            3500   
47   8  sa_LtoyRMVW7Uk9y5gfL2  Intranet      MXN            3500   
48   7  sa_xPP4vqQIdjOH2pWmoT  Intranet      MXN            3500   
49   6  sa_yy5L90ydhP4sdcfOwb  Intranet      MXN            3500   
50   5  sa_Fg0vjCIm7ELfWs7RAM  Intranet      MXN            3500   
51   4  sa_hrutNpyF2ohHq6H39H  Intranet      MXN            3500   
52   3  sa_Hq4vOldCVlyd8qiJbM  Intranet      MXN            3500   
53   2  sa_XBXzMml9W1iSudlkOg  Intranet      MXN            3500   
54   1  sa_6sMjmqdyltYkKw9rkL  Intranet      MXN            3500   

    property_amount  property_discount_amount  down_payment_amount  \
0            517916                   51791.6            186449.00   
1            850360                   42518.0          

### Customers

In [95]:
def GetCustomers(token: str):
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"  # Ensures the server knows you expect JSON
    }
    url = "https://dashboard.colorines.paitesting.com/api/v1/customers"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

def GetCustomerID(df, email):
    df_filtered = df[df['email'] == email]
    if (len(df_filtered)>0):
        return df_filtered.iloc[0].id
    else:
        return None

In [73]:
customers = GetCustomers(token)
GetCustomerID(customers, 'admin@platform.com')

1

## Clean functions

In [90]:
def CleanProjects(df):
    df_clean = df[['name', 'development_name', 'active_steps']]
    df_clean = df_clean.rename(columns={'name': 'nombre_proyecto', 'development_name': 'nombre_desarrollo', 'active_steps': 'etapa'})
    return df_clean.to_json(orient='records', force_ascii=False)

def CleanProperties(df):
    df_clean = df[['alias', 'status', 'project_name', 'development_name', 'step_number', 'total_amount_mxn', 'total_amount_usd']]
    
    df_clean = df_clean.rename(columns={'alias': 'nombre_propiedad',
                                        'project_name': 'nombre_proyecto',
                                        'development_name': 'nombre_desarrollo',
                                        'step_number': 'etapa',
                                        'total_amount_mxn': 'precio_MXN',
                                        'total_amount_usd': 'precio_USD'
                                       })
    return df_clean.to_json(orient='records', force_ascii=False)

def CleanSalesByCustomer(df):
    df_clean = df[['property_name', 'currency', 'reserve_amount', 'property_amount', 'property_discount_amount', 'down_payment_amount', 'down_payment_percent', 'number_installments', 'installments_amount', 'installments_individual_amount', 'sqft_price_amount', 'status', 'amount_total', 'amount_paid', 'amount_remaining', 'next_payment_date', 'advance_percent', 'overdue_installments_count', 'overdue_installments_amount']]
    return df_clean.to_json(orient='records', force_ascii=False)

In [85]:
sales.head(2)
sales

id              custom_id   channel currency  reserve_amount  \
0  55  sa_4HKug50MakfSTRruEU  Intranet      MXN            3500   
1  54  sa_zQiJaxCxPXxgTlZUVR  Intranet      MXN            3500   

   property_amount  property_discount_amount  down_payment_amount  \
0           517916                   51791.6             186449.0   
1           850360                   42518.0                  0.0   

   down_payment_percent  down_payment_installments  number_installments  \
0                    40                          5                   24   
1                     0                          1                   57   

   installments_amount  installments_individual_amount  sqft_price_amount  \
0             279675.4                         11653.0             1260.0   
1             807842.0                         14172.0             1330.0   

                                                                  status  \
0  {'name': 'Contrato firmado', 'short_code': 'contract', 'percent': 75}   
1    {'name': 'Documentación', 'short_code': 'documents', 'percent': 50}   

   absolute_value  amount_total  amount_paid  amount_remaining  \
0        466124.4      466124.4      46612.0          419512.4   
1        807842.0      807842.0          0.0          807842.0   

            first_payment_date  recurring_payday            next_payment_date  \
0  2024-07-02T06:00:00.000000Z                 2  2024-07-02T06:00:00.000000Z   
1  2024-07-02T06:00:00.000000Z                 2  2024-07-02T06:00:00.000000Z   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            prices  \
0  [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}]   
1  [{'type': 'property', 'amount': 1400, 'amount_with_decimals': '1,400.00', 'amount_without_decimals': '1400', 'total_amount': 1400, 'total_amount_with_decimals': '1,400.00', 'total_amount_without_decimals': '1400', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}, {'type': 'reserve', 'amount': 3500, 'amount_with_decimals': '3,500.00', 'amount_without_decimals': '3500', 'total_amount': 3500, 'total_amount_with_decimals': '3,500.00', 'total_amount_without_decimals': '3500', 'currency_id': 1, 'currency_symbol': '$', 'currency_code': 'MXN'}]   

   bank_account_id  \
0               11   
1               11   

                                                                                                                                                           bank_account  \
0  {'id': 11, 'name': 'SS P Intercam', 'bank': 'INTERCAM', 'clabe': '136691109833600123', 'currency': 'MXN', 'stripe_account': 'acct_1NOqogIDxmEzZD2X', 'is_active': 1}   
1  {'id': 11, 'name': 'SS P Intercam', 'bank': 'INTERCAM', 'clabe': '136691109833600123', 'currency': 'MXN', 'stripe_account': 'acct_1NOqogIDxmEzZD2X', 'is_active': 1}   

   advance_percent                 reserve_date unlock_date  able_to_unlock  \
0         9.999906  2024-06-20T16:01:30.000000Z        None           False   
1         0.000000  2024-

## Tools

In [37]:
@tool
def get_active_properties(token: str) -> str:
    """
    Uses the token to fetch the active properties from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    properties_df = GetProperties(token)
    properties_df = properties_df[properties_df['is_active'] == True]
    

    response = requests.get(url, headers=headers, params=params)
    # Check if the response was successful
    if response.status_code == 200:
        try:
            return json.dumps(response.json())  # Convert JSON response to string
        except ValueError:  # Catch JSON errors
            return json.dumps({"error": "Error parsing JSON"})
    else:
        return json.dumps({"error": f"Received status code {response.status_code}"})



@tool
def get_last_sale(token: str) -> str:
    """
    Uses the token to fetch the last sale from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params = {"per_page": 1, "sort": "-created_at"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})



### Properties, developments and Steps

In [38]:
@tool
def get_active_projects(token: str) -> str:   
    """
    Uses the token to fetch the active (available) projects from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    df = GetProjects(token)
    pre_df = PreprocessProjects(df)
    return CleanProjects(pre_df)
    
@tool
def get_active_properties(token: str)-> str:
    """
    Uses the token to fetch the active (available) properties from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    
    df = GetProperties(token)
    pre_df = PreprocessProperties(df)
    return CleanProperties(pre_df)



In [5]:
print(get_active_projects.name)
print(get_active_projects.description)
print(get_active_projects.args)

get_active_projects
Uses the token to fetch the active (available) projects from colorines API.

Args:
    token (str): The authorization token required to access the API.

Returns:
    str: The JSON response from the API as a string.
{'token': {'title': 'Token', 'type': 'string'}}


In [17]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

### Sales

In [100]:
@tool
def count_sales_by_status(token: str, status: str)-> int:
    """
    Count the number of sales the user has with a specific status.
    
    
    Args:
        token (str): The authorization token required to access the API.
        status (str): The status to filter the API request. Acceptable status by the API: reserved, documents, contract, active, canceled, overdue, finalized, deeded.

    Returns:
        int: The number of sales with the requested status.
    """
    
    df = GetSales(token=token, status=status)
    count = len(df)
    return count

@tool
def get_sales_by_customer(token: str, email: str) -> str:
    """
    Retrieves and returns a list of sales associated with a customer identified by their email.

    This function performs the following steps:
    1. Uses the provided email to search for the customer ID.
    2. Retrieves the sales attached to that customer ID.
    3. Returns relevant sales information in JSON format.

    Args:
        token (str): The authorization token required to access the API.
        email (str): The email address used to identify the customer.

    Returns:
        str: A JSON response containing information about sales made to the customer.
             The JSON includes details about each sale that will be listed to the user by the agent.
    """
    # Function implementation here
    customers = GetCustomers(token)
    customer_id = GetCustomerID(customers, email=email)
    if customer_id:
        df = GetSales(token=token, customer_id=customer_id)
        df = PreprocessSales(test_sales)
        return CleanSalesByCustomer(df)
    else:
        return 'No se encontró ningún cliente con ese correo electrónico, pregunta al cliente si el correo es correcto'




## Test agent

In [102]:
    
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who will use the {token} to use the tools to bring information to the user"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Example of integrating with langchain agent framework
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Add the `get_sales` function to the list of tools
tools_api = [get_last_sale, get_active_projects, count_sales_by_status, get_sales_by_customer] 

# Create the agent
agent = create_tool_calling_agent(llm, tools_api, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_api, verbose=True)

# Example usage:
# token = 'your_actual_token_here'
# base_url = 'https://api.yourdomain.com'


In [103]:
TEST_MAIL

'admin@platform.com'

In [ ]:
input_data = {
    "input": "hay ventas del cliente admin@platform.com?",
    "token": token
}

# Invoke the agent
agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...
